In [2]:
!pip install datasets sentence-transformers
!pip install --upgrade datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 125.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 40.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 107.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvji

  Using cached datasets-4.0.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
Using cached datasets-4.0.0-py3-none-any.whl (494 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.


In [3]:
from sentence_transformers import SentenceTransformer, SentencesDataset, InputExample
import torch
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from datasets import load_dataset
from torch.utils.data import DataLoader
from sentence_transformers.losses import CosineSimilarityLoss
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

In [4]:
# call English STS-Benchmark
raw = load_dataset("glue", "stsb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/502k [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/151k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/114k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5749 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1500 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1379 [00:00<?, ? examples/s]

In [5]:
from sentence_transformers import InputExample

def to_examples(split):
    examples = []
    for row in raw[split]:
        examples.append(
            InputExample(
                texts=[row["sentence1"], row["sentence2"]],
                label=float(row["label"])))
    return examples

train_examples = to_examples("train")
dev_examples   = to_examples("validation")



In [6]:


model = SentenceTransformer("sentence-transformers/stsb-roberta-large")
for param in model.parameters():
    param.requires_grad = False

transformer_module = model._modules['0']

for param in transformer_module.auto_model.embeddings.parameters():
    param.requires_grad = True

for layer in transformer_module.auto_model.encoder.layer[-3:]:
    for param in layer.parameters():
        param.requires_grad = True

train_dataset    = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)
train_loss       = CosineSimilarityLoss(model)

dev_evaluator = EmbeddingSimilarityEvaluator.from_input_examples(
    dev_examples, name="sts-dev")




modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/2.00 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [7]:
import os
os.environ["WANDB_DISABLED"] = "true"



In [8]:


best_spearman = -1.0
no_improve    = 0
patience      = 5

for epoch in range(1, 11):
    model.fit(
        train_objectives=[(train_dataloader, train_loss)],
        epochs=1,
        evaluation_steps=0,
        save_best_model=True,
        output_path="output/sts-3layers",
        evaluator=dev_evaluator,)

    results = dev_evaluator(model, output_path=None, epoch=epoch, steps=0)

    spearman_key = next(k for k in results if "spearman" in k)
    curr = results[spearman_key]

    if curr > best_spearman + 1e-4:
        best_spearman, no_improve = curr, 0
    else:
        no_improve += 1

    if no_improve >= patience:
        print(f"⏹ Early stopping at epoch {epoch}, best Spearman={best_spearman:.4f}")
        break


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Computing widget examples:   0%|          | 0/1 [00:00<?, ?example/s]

Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.869700,0.875324


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.842996,0.857632


...

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.767513,0.817409


Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss,Validation Loss,Sts-dev Pearson Cosine,Sts-dev Spearman Cosine
360,No log,No log,0.770046,0.819146


⏹ Early stopping at epoch 6, best Spearman=0.8753


In [9]:
import os, time, requests, pandas as pd
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

API_KEY = os.getenv("RAPIDAPI_KEY") or "ad973560c8msh96909b7fb3cc3fdp1c1065jsn5a49e6b93474"
HEADERS = {
    "X-RapidAPI-Key":  API_KEY,
    "X-RapidAPI-Host": "seeking-alpha.p.rapidapi.com",
}

session = requests.Session()
retry_cfg = Retry(
    total=5,
    backoff_factor=1,
    status_forcelist=[429, 500, 502, 503, 504],
    allowed_methods=["GET"],
)
session.mount("https://", HTTPAdapter(max_retries=retry_cfg))


def _respect_rate_limit(resp):
    """Sleep if remaining-requests header is low."""
    left = resp.headers.get("X-RateLimit-Requests-Remaining")
    reset = resp.headers.get("X-RateLimit-Requests-Reset")
    if left is not None and reset is not None:
        left, reset = int(left), int(reset)
        if left < 2:
            sleep_s = max(reset - int(time.time()), 1)
            print(f"[rate-limit] only {left} calls left → sleeping {sleep_s}s")
            time.sleep(sleep_s)

# extracting the news with particular symbols

def fetch_symbol_news(symbol: str,
                      max_items: int = 200,
                      page_size: int = 40,
                      since: int = 0,
                      until: int = 0) -> pd.DataFrame:
    url   = "https://seeking-alpha.p.rapidapi.com/news/v2/list-by-symbol"
    items = []
    page  = 1

    while len(items) < max_items:
        p = {"id": symbol, "size": page_size, "number": page}
        if since: p["since"] = since
        if until: p["until"] = until

        r = session.get(url, headers=HEADERS, params=p, timeout=20)
        r.raise_for_status()
        batch = r.json().get("data", [])
        if not batch:
            break

        items.extend(batch)
        if len(batch) < page_size:
            break

        page += 1
        time.sleep(0.25 + 0.25 * os.urandom(1)[0] / 255)

    if not items:
        return pd.DataFrame(columns=["title", "publishOn", "symbol"])

    df = pd.json_normalize(items[:max_items])

    for c in ["attributes.title", "attributes.headline", "attributes.teaser"]:
        if c in df.columns:
            df = df.rename(columns={c: "title"})
            break
    if "title" not in df.columns:
        df["title"] = ""

    if "publishOn" in df.columns:
        df["publishOn"] = pd.to_datetime(df["publishOn"], utc=True)
    elif "publishedAt" in df.columns:
        df["publishOn"] = pd.to_datetime(df["publishedAt"], utc=True)

    df["symbol"] = symbol
    return df



In [ ]:
# collect the return data on the publish day of news and convert to 1,0

def label_news_df(df: pd.DataFrame, symbol: str) -> pd.DataFrame:
    df = df.copy()
    if "attributes.publishOn" in df.columns:
        df = df.rename(columns={"attributes.publishOn": "publishOn"})


    df["publishOn"] = (
        pd.to_datetime(df["publishOn"], utc=True, errors="coerce")
          .dt.tz_localize(None)
    )
    df = df.dropna(subset=["publishOn"])

    # 1) build price window
    earliest = df["publishOn"].dt.date.min()
    latest   = df["publishOn"].dt.date.max()
    start_dt = (pd.Timestamp(earliest) - pd.Timedelta(days=7)).strftime("%Y-%m-%d")
    end_dt   = (pd.Timestamp(latest)   + pd.Timedelta(days=7)).strftime("%Y-%m-%d")

    px = yf.download(symbol, start=start_dt, end=end_dt, interval="1d")

    # extract the Close price
    closes = px["Close"]
    if isinstance(closes, pd.DataFrame):
        closes = closes.squeeze()

    # normalise to midnight dates
    closes.index = closes.index.tz_localize(None).normalize()
    trading_dates = closes.index

    ret_vals, labels = [], []
    for ts in df["publishOn"]:
        pub = pd.to_datetime(ts).tz_localize(None).normalize()

        prev_days = trading_dates[trading_dates < pub]
        next_days = trading_dates[trading_dates > pub]
        if prev_days.empty or next_days.empty:
            ret_vals.append(np.nan)
            labels.append(np.nan)
            continue

        d0, d1 = prev_days.max(), next_days.min()
        p0, p1 = closes.at[d0], closes.at[d1]

        if pd.isna(p0) or pd.isna(p1) or p0 == 0:
            ret_vals.append(np.nan)
            labels.append(np.nan)
            continue

        r = p1 / p0 - 1
        ret_vals.append(r)
        labels.append(int(r > 0))

    df["return_1d"] = ret_vals
    df["label"]     = labels
    df = df.dropna(subset=["label"]).reset_index(drop=True)
    return df



In [ ]:
from google.colab import drive
import pickle, os

drive.mount('/content/drive', force_remount=True)

path = '/content/drive/MyDrive/colab_saves/company_dfs.pkl'

!ls -lh "{path}"
with open(path, 'rb') as f:
    company_dfs = pickle.load(f)
print(type(company_dfs))
print(company_dfs.keys())

In [14]:


symbols_2 = ["C", "GS", "MS", "USB"]

company_df_2 = {
    sym: fetch_symbol_news(sym, max_items=400, since=0, until=0)
    for sym in symbols_2}



In [15]:


symbols_3 = ["AAPL", "MSFT", "AMZN", "GOOGL"]

company_df_3 = {
    sym: fetch_symbol_news(sym, max_items=400, since=0, until=0)
    for sym in symbols_3}



In [16]:

symbols_4 = ["META", "NVDA", "TSLA", "NFLX"]

company_df_4 = {
    sym: fetch_symbol_news(sym, max_items=400, since=0, until=0)
    for sym in symbols_4}



In [19]:
import pickle
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
path = '/content/drive/MyDrive/colab_saves/company_dfs.pkl'


with open(path, 'rb') as f:
    company_dfs = pickle.load(f)

new_groups = {
    'group2': company_df_2,
    'group3': company_df_3,
    'group4': company_df_4,
}

for key, df in new_groups.items():
        company_dfs[key] = df

with open(path, 'wb') as f:
    pickle.dump(company_dfs, f)


with open(path, 'rb') as f:
    updated = pickle.load(f)
print("Updated keys:", list(updated.keys()))


KeyboardInterrupt: 